<a href="https://colab.research.google.com/github/Vaibhav21pandit/nn_in_pytorch/blob/main/CNN_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
from torchvision import datasets
from torch.utils.data import Dataset,DataLoader

In [17]:
class LeNet(nn.Module):
  def __init__(self):
    super(LeNet,self).__init__()
    self.pool=nn.AvgPool2d(kernel_size=(2,2),stride=(2,2))
    self.conv1=nn.Conv2d(in_channels=3,out_channels=6,kernel_size=(5,5),stride=(1,1),padding=(0,0))
    self.conv2=nn.Conv2d(in_channels=6,out_channels=16,kernel_size=(5,5),stride=(1,1),padding=(0,0))
    self.conv3=nn.Conv2d(in_channels=16,out_channels=120,kernel_size=(5,5),stride=(1,1),padding=(0,0))
    self.fc1=nn.Linear(in_features=120,out_features=84)
    self.fc2=nn.Linear(in_features=84,out_features=10)

  def forward(self,input):
    x=torch.relu(self.conv1(input))
    x=self.pool(x)
    x=torch.relu(self.conv2(x))
    x=self.pool(x)
    x=torch.relu(self.conv3(x))
    x=x.reshape(x.shape[0],-1)
    x=torch.relu(self.fc1(x))
    x=self.fc2(x)
    return x

model=LeNet()




In [18]:
input=torch.rand((1,3,32,32))
print(model(input).shape)

torch.Size([1, 10])
